# Standard U-Net Training Notebook
## Fetal Head Segmentation in Ultrasound Images

This notebook implements a **standard U-Net** baseline model with:
1. Standard encoder-decoder architecture with skip connections
2. ReLU activations in all convolutional layers
3. Padded convolutions (`padding='same'`) to maintain spatial dimensions
4. Image normalization (dividing by 255.0)
5. Adam optimizer with learning rate 1e-4
6. **Dice Loss function** (optimized for class imbalance)

## 1. Setup and Imports

**Environment Detection:**
- Automatically detects if running on Kaggle or locally
- On Kaggle: Uses `/kaggle/input/fetal-head-segmentation` for read-only data
- On Kaggle: Uses `/kaggle/working` for writable outputs (cache, checkpoints, logs)
- Locally: Uses project structure as-is

In [ ]:
import os
import sys
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Detect environment and set project root
if os.path.exists('/kaggle/input'):
    # Running on Kaggle
    print("Running on Kaggle")
    project_root = Path('/kaggle/input/fetal-head-segmentation')
    print(f"Project root: {project_root}")
    print(f"Project root exists: {project_root.exists()}")
    
    if not (project_root / 'accuracy_focus').exists():
        raise RuntimeError(
            f"'accuracy_focus' folder not found in {project_root}\n"
            f"Please ensure your GitHub repository was added correctly as a Kaggle dataset."
        )
else:
    # Running locally
    print("Running locally")
    current = Path(os.getcwd())
    project_root = None
    
    # Try to find project root by going up
    for parent in [current] + list(current.parents):
        if (parent / 'accuracy_focus').exists():
            project_root = parent
            break
    
    if project_root is None:
        raise RuntimeError(
            f"Cannot find project root with 'accuracy_focus' folder.\n"
            f"Current directory: {os.getcwd()}"
        )

sys.path.insert(0, str(project_root))

# Import from project structure (updated paths)
from accuracy_focus.standard_unet.src.models.standard_unet import StandardUNet
from accuracy_focus.standard_unet.src.losses import DiceLoss
from shared.src.data.cached_dataset import CachedHC18Dataset
from shared.src.metrics.segmentation_metrics import dice_coefficient, iou_score, pixel_accuracy
from shared.src.utils.visualization import save_prediction_grid, visualize_sample

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. Load Configuration

In [ ]:
# Load configuration from new location
config_path = project_root / 'accuracy_focus' / 'standard_unet' / 'configs' / 'standard_unet_config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# On Kaggle, adjust output paths to /kaggle/working (input is read-only)
if os.path.exists('/kaggle/input'):
    print("Adjusting paths for Kaggle environment...")
    working_dir = Path('/kaggle/working')
    
    # Keep cache pointing to read-only input (already preprocessed)
    # Only update output directories to writable location
    config['logging']['checkpoint_dir'] = str(working_dir / 'results' / 'checkpoints')
    config['logging']['log_dir'] = str(working_dir / 'results' / 'logs')
    config['logging']['prediction_dir'] = str(working_dir / 'results' / 'predictions')
    config['logging']['visualization_dir'] = str(working_dir / 'results' / 'visualizations')
    
    print(f"  Cache dir: {config['data']['cache_dir']} (read-only, using existing)")
    print(f"  Checkpoint dir: {config['logging']['checkpoint_dir']}")
else:
    # Running locally - cache should already point to shared/preprocessed_data
    print(f"  Cache dir: {config['data']['cache_dir']}")

print("Configuration loaded:")
print(f"  Model: {config['model']['name']}")
print(f"  Base Filters: {config['model']['base_filters']}")
print(f"  Learning Rate: {config['training']['optimizer']['lr']}")
print(f"  Loss Function: {config['loss']['name']}")
print(f"  Batch Size: {config['training']['batch_size']}")
print(f"  Epochs: {config['training']['num_epochs']}")

## 3. Initialize Model

The Standard U-Net has:
- **Encoder**: 4 downsampling blocks (64 → 128 → 256 → 512 channels)
- **Bottleneck**: 1024 channels
- **Decoder**: 4 upsampling blocks (512 → 256 → 128 → 64 channels)
- **Skip Connections**: Concatenate encoder features to decoder
- **Activation**: ReLU in all conv layers
- **Output**: Sigmoid for binary segmentation

In [ ]:
# Set device
device = torch.device(config['device'] if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize model
model = StandardUNet(
    in_channels=config['model']['in_channels'],
    out_channels=config['model']['out_channels'],
    base_filters=config['model']['base_filters']
)
model = model.to(device)

# Model summary
print(f"\nStandard U-Net Architecture:")
print(f"  Total trainable parameters: {model.count_parameters():,}")
print(f"  Input channels: {config['model']['in_channels']}")
print(f"  Output channels: {config['model']['out_channels']}")
print(f"  Base filters: {config['model']['base_filters']}")

# Test forward pass
test_input = torch.randn(1, 1, 256, 256).to(device)
test_output = model(test_input)
print(f"\nTest forward pass:")
print(f"  Input shape: {test_input.shape}")
print(f"  Output shape: {test_output.shape}")
print(f"  Output range: [{test_output.min().item():.4f}, {test_output.max().item():.4f}]")

## 4. Setup Loss and Optimizer

- **Loss**: Dice Loss (optimized for class imbalance)
- **Optimizer**: Adam with learning rate 1e-4
- **Scheduler**: ReduceLROnPlateau

In [ ]:
# Loss function (Dice Loss)
criterion = DiceLoss(smooth=1.0)
print(f"Loss Function: Dice Loss")
print(f"  Smooth parameter: 1.0")

# Optimizer (Adam with lr=1e-4)
optimizer_config = config['training']['optimizer']
optimizer = optim.Adam(
    model.parameters(),
    lr=optimizer_config['lr'],
    betas=tuple(optimizer_config['betas']),
    eps=optimizer_config['eps'],
    weight_decay=optimizer_config['weight_decay']
)
print(f"Optimizer: Adam")
print(f"  Learning rate: {optimizer_config['lr']}")

# Learning rate scheduler
scheduler_config = config['training']['scheduler']
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode=scheduler_config['mode'],
    factor=scheduler_config['factor'],
    patience=scheduler_config['patience'],
    min_lr=scheduler_config['min_lr']
)
print(f"Scheduler: ReduceLROnPlateau")
print(f"  Mode: {scheduler_config['mode']}")
print(f"  Factor: {scheduler_config['factor']}")
print(f"  Patience: {scheduler_config['patience']}")

## 5. Prepare Data Loaders

**Preprocessing**:
- Images normalized by dividing by 255.0
- Resized to 256×256 pixels
- Converted to tensors

**Augmentation** (training only):
- Horizontal flip (p=0.5)
- Rotation (±20°, p=0.5)
- Scale (±10%, p=0.5)
- Translate (±10%, p=0.5)

In [ ]:
data_config = config['data']
training_config = config['training']

# Build paths - use config paths directly if already absolute (Kaggle), otherwise join with project_root
def get_path(config_path):
    """Helper to handle both absolute and relative paths"""
    p = Path(config_path)
    if p.is_absolute():
        return str(p)
    else:
        return str(project_root / config_path)

# Check if preprocessed cache exists
train_cache_path = get_path(data_config['cache_dir'] + '/train_cache')
val_cache_path = get_path(data_config['cache_dir'] + '/val_cache')

print("\nChecking for existing preprocessed data...")
print(f"  Train cache: {train_cache_path}")
print(f"    Exists: {Path(train_cache_path).exists()}")
if Path(train_cache_path).exists():
    metadata_file = Path(train_cache_path) / 'metadata.pkl'
    print(f"    Metadata: {metadata_file.exists()}")

print(f"  Val cache: {val_cache_path}")
print(f"    Exists: {Path(val_cache_path).exists()}")
if Path(val_cache_path).exists():
    metadata_file = Path(val_cache_path) / 'metadata.pkl'
    print(f"    Metadata: {metadata_file.exists()}")

# Train dataset - will use existing cache if available
train_dataset = CachedHC18Dataset(
    image_dir=get_path(data_config['train_images']),
    mask_dir=get_path(data_config['train_masks']),
    cache_dir=train_cache_path,
    force_rebuild=False  # Explicitly use existing cache if available
)

# Validation dataset - will use existing cache if available
val_dataset = CachedHC18Dataset(
    image_dir=get_path(data_config['val_images']),
    mask_dir=get_path(data_config['val_masks']),
    cache_dir=val_cache_path,
    force_rebuild=False  # Explicitly use existing cache if available
)

# Data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=training_config['batch_size'],
    shuffle=True,
    num_workers=training_config['num_workers'],
    pin_memory=training_config['pin_memory']
)

val_loader = DataLoader(
    val_dataset,
    batch_size=training_config['batch_size'],
    shuffle=False,
    num_workers=training_config['num_workers'],
    pin_memory=training_config['pin_memory']
)

print(f"\nDatasets:")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Validation samples: {len(val_dataset)}")
print(f"  Batch size: {training_config['batch_size']}")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Normalization: Divide by 255.0")

## 6. Visualize Sample Data

In [ ]:
# Get a batch of training data
sample_images, sample_masks = next(iter(train_loader))

print(f"Sample batch:")
print(f"  Images shape: {sample_images.shape}")
print(f"  Masks shape: {sample_masks.shape}")
print(f"  Image range: [{sample_images.min():.4f}, {sample_images.max():.4f}]")
print(f"  Mask range: [{sample_masks.min():.4f}, {sample_masks.max():.4f}]")

# Visualize first sample
visualize_sample(sample_images[0], sample_masks[0])

## 7. Training Functions

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch):
    """
    Train for one epoch
    """
    model.train()
    running_loss = 0.0
    
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1} [Train]", leave=False)
    for batch_idx, (images, masks) in enumerate(pbar):
        images = images.to(device)
        masks = masks.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        
        # Calculate loss (Dice Loss)
        loss = criterion(outputs, masks)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Update statistics
        running_loss += loss.item()
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})
    
    epoch_loss = running_loss / len(dataloader)
    return epoch_loss


def validate(model, dataloader, criterion, device, epoch):
    """
    Validate the model
    """
    model.eval()
    running_loss = 0.0
    dice_scores = []
    iou_scores = []
    pa_scores = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1} [Val]", leave=False)
        for batch_idx, (images, masks) in enumerate(pbar):
            images = images.to(device)
            masks = masks.to(device)
            
            # Forward pass
            outputs = model(images)
            
            # Calculate loss
            loss = criterion(outputs, masks)
            running_loss += loss.item()
            
            # Calculate metrics
            preds = (outputs > 0.5).float()
            
            for i in range(images.size(0)):
                dice = dice_coefficient(preds[i], masks[i])
                iou = iou_score(preds[i], masks[i])
                pa = pixel_accuracy(preds[i], masks[i])
                
                dice_scores.append(dice.item())
                iou_scores.append(iou.item())
                pa_scores.append(pa.item())
            
            pbar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'dice': f"{np.mean(dice_scores):.4f}"
            })
    
    # Calculate average metrics
    val_loss = running_loss / len(dataloader)
    val_dice = np.mean(dice_scores)
    val_iou = np.mean(iou_scores)
    val_pa = np.mean(pa_scores)
    
    return val_loss, val_dice, val_iou, val_pa


print("Training functions defined!")

## 8. Training Loop

In [ ]:
# Training configuration
num_epochs = config['training']['num_epochs']
patience = config['training']['early_stopping_patience']

# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'val_dice': [],
    'val_iou': [],
    'val_pa': [],
    'lr': []
}

best_dice = 0.0
epochs_without_improvement = 0

print(f"{'='*60}")
print(f"Starting Training - Standard U-Net")
print(f"{'='*60}")
print(f"Epochs: {num_epochs}")
print(f"Loss Function: Dice Loss")
print(f"Optimizer: Adam (lr=1e-4)")
print(f"Early Stopping Patience: {patience}")
print(f"{'='*60}\n")

In [ ]:
# Main training loop
for epoch in range(num_epochs):
    # Train
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch)
    
    # Validate
    val_loss, val_dice, val_iou, val_pa = validate(model, val_loader, criterion, device, epoch)
    
    # Update learning rate
    scheduler.step(val_dice)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Update history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_dice'].append(val_dice)
    history['val_iou'].append(val_iou)
    history['val_pa'].append(val_pa)
    history['lr'].append(current_lr)
    
    # Print epoch summary
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Loss:   {val_loss:.4f}")
    print(f"  Val Dice:   {val_dice:.4f}")
    print(f"  Val IoU:    {val_iou:.4f}")
    print(f"  Val PA:     {val_pa:.4f}")
    print(f"  LR:         {current_lr:.6f}")
    
    # Check for improvement
    is_best = val_dice > best_dice
    if is_best:
        best_dice = val_dice
        epochs_without_improvement = 0
        
        # Save best model - use get_path helper for both local and Kaggle
        checkpoint_dir = Path(get_path(config['logging']['checkpoint_dir']))
        checkpoint_dir.mkdir(parents=True, exist_ok=True)
        best_model_path = checkpoint_dir / 'best_model.pth'
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_dice': best_dice,
            'history': history,
            'config': config
        }, best_model_path)
        
        print(f"  → Saved best model (Dice: {best_dice:.4f})")
    else:
        epochs_without_improvement += 1
    
    # Early stopping
    if epochs_without_improvement >= patience:
        print(f"\n{'='*60}")
        print(f"Early stopping triggered after {epoch+1} epochs")
        print(f"Best Dice Score: {best_dice:.4f}")
        print(f"{'='*60}")
        break

# Final summary
print(f"\n{'='*60}")
print(f"Training Completed!")
print(f"{'='*60}")
print(f"Best Validation Dice: {best_dice:.4f}")
print(f"Best Validation IoU:  {max(history['val_iou']):.4f}")
print(f"Best Validation PA:   {max(history['val_pa']):.4f}")
print(f"{'='*60}\n")

## 9. Plot Training Curves

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Loss curves
axes[0, 0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0, 0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Loss (Dice)', fontsize=12)
axes[0, 0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Dice coefficient
axes[0, 1].plot(history['val_dice'], label='Val Dice', color='green', linewidth=2)
axes[0, 1].axhline(y=best_dice, color='red', linestyle='--', label=f'Best: {best_dice:.4f}')
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Dice Coefficient', fontsize=12)
axes[0, 1].set_title('Validation Dice Coefficient', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# IoU
axes[1, 0].plot(history['val_iou'], label='Val IoU', color='orange', linewidth=2)
axes[1, 0].axhline(y=max(history['val_iou']), color='red', linestyle='--', 
                   label=f"Best: {max(history['val_iou']):.4f}")
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('IoU Score', fontsize=12)
axes[1, 0].set_title('Validation IoU Score', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Learning rate
axes[1, 1].plot(history['lr'], label='Learning Rate', color='red', linewidth=2)
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Learning Rate', fontsize=12)
axes[1, 1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_yscale('log')

plt.tight_layout()
plt.show()

# Save figure - use get_path helper
log_dir = Path(get_path(config['logging']['log_dir']))
log_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(log_dir / 'training_curves.png', dpi=150, bbox_inches='tight')
print(f"Training curves saved to {log_dir / 'training_curves.png'}")

## 10. Visualize Predictions

In [ ]:
# Load best model - use get_path helper
checkpoint_path = Path(get_path(config['logging']['checkpoint_dir'])) / 'best_model.pth'
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Loaded best model from epoch {checkpoint['epoch']+1}")
print(f"Best Dice Score: {checkpoint['best_dice']:.4f}")

# Get validation samples
val_images, val_masks = next(iter(val_loader))
val_images = val_images.to(device)

# Generate predictions
with torch.no_grad():
    val_preds = model(val_images)
    val_preds = (val_preds > 0.5).float()

# Visualize predictions
num_samples = min(4, len(val_images))
fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4 * num_samples))

if num_samples == 1:
    axes = axes.reshape(1, -1)

for i in range(num_samples):
    # Move to CPU and convert to numpy
    img = val_images[i, 0].cpu().numpy()
    mask = val_masks[i, 0].numpy()
    pred = val_preds[i, 0].cpu().numpy()
    
    # Calculate metrics for this sample
    dice = dice_coefficient(val_preds[i].cpu(), val_masks[i].to(device)).item()
    iou = iou_score(val_preds[i].cpu(), val_masks[i].to(device)).item()
    
    # Input image
    axes[i, 0].imshow(img, cmap='gray')
    axes[i, 0].set_title('Input Image', fontsize=12, fontweight='bold')
    axes[i, 0].axis('off')
    
    # Ground truth
    axes[i, 1].imshow(mask, cmap='gray')
    axes[i, 1].set_title('Ground Truth', fontsize=12, fontweight='bold')
    axes[i, 1].axis('off')
    
    # Prediction
    axes[i, 2].imshow(pred, cmap='gray')
    axes[i, 2].set_title(f'Prediction\nDice: {dice:.4f} | IoU: {iou:.4f}', 
                         fontsize=12, fontweight='bold')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

# Save predictions - use get_path helper
pred_dir = Path(get_path(config['logging']['prediction_dir']))
pred_dir.mkdir(parents=True, exist_ok=True)
save_prediction_grid(val_images[:4].cpu(), val_masks[:4], val_preds[:4].cpu(), 
                    str(pred_dir / 'sample_predictions.png'), num_samples=4)
print(f"Sample predictions saved to {pred_dir / 'sample_predictions.png'}")

## 11. Summary

### Key Points:

1. **Model Architecture**: Standard U-Net with skip connections
2. **Activation**: ReLU in all convolutional layers
3. **Padding**: `padding=1` (same) to maintain spatial dimensions
4. **Normalization**: Images divided by 255.0
5. **Optimizer**: Adam with learning rate 1e-4
6. **Loss Function**: Dice Loss (handles class imbalance)

### Results:

View the results above and compare with your improved U-Net baseline.

### Why Dice Loss?

Dice Loss directly optimizes for segmentation overlap, making it more effective than BCE for:
- **Class imbalance**: Most pixels are background in medical images
- **Small objects**: Fetal head is a small region compared to background
- **Direct metric optimization**: Loss matches the evaluation metric (Dice coefficient)